<a href="https://colab.research.google.com/github/anilbhatt1/Deep_Learning_EVA4_Phase1/blob/master/Master_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mounting the drive and copying the code from github to drive
from google.colab import drive 
drive.mount('/content/gdrive')
%cd gdrive/My Drive/project_folder
! ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/project_folder
Untitled0.ipynb


In [2]:
!git clone https://github.com/anilbhatt1/Deep_Learning_EVA4_Phase1

Cloning into 'Deep_Learning_EVA4_Phase1'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 543 (delta 96), reused 0 (delta 0), pack-reused 389
Receiving objects: 100% (543/543), 4.16 MiB | 14.70 MiB/s, done.
Resolving deltas: 100% (333/333), done.


In [3]:
%cd Deep_Learning_EVA4_Phase1/S7_CIFAR10_Modular
! ls

/content/gdrive/My Drive/project_folder/Deep_Learning_EVA4_Phase1/S7_CIFAR10_Modular
Master_Model.py  models  README.md


In [0]:
from __future__ import print_function
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from models import S7_Datatransforms,S7_Model,S7_Dataloaders,S7_Train_Losses,S7_Test_losses

In [5]:
data_loader = S7_Dataloaders.DataLoader(shuffle=True,batch_size=128,seed=1)

mean = (0.49139968, 0.48215841, 0.44653091)
stdev = (0.24703223, 0.24348513, 0.26158784)
transform_instance  = S7_Datatransforms.Transforms_custom(normalize=True, mean=mean, stdev=stdev)

before_norm_list = [transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
               transforms.RandomRotation((-7,7),fill=(1,))]
               
test_transforms   = transform_instance.test_transforms()
train_transforms  = transform_instance.train_transforms(before_norm = None)

train_data = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test_data  = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
trainloader = data_loader.load(train_data)
testloader  = data_loader.load(test_data)

In [7]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = S7_Model.Net()
model.to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          18,432
              ReLU-6           [-1, 64, 32, 32]               0
       BatchNorm2d-7           [-1, 64, 32, 32]             128
           Dropout-8           [-1, 64, 32, 32]               0
         MaxPool2d-9           [-1, 64, 16, 16]               0
           Conv2d-10           [-1, 32, 16, 16]           2,048
           Conv2d-11           [-1, 64, 16, 16]          18,432
             ReLU-12           [-1, 64, 16, 16]               0
      BatchNorm2d-13           [-1, 64, 16, 16]             128
          Dropout-14           [-1

In [0]:
train_loss = S7_Train_Losses.Train_loss()
test_loss  = S7_Test_losses.Test_loss()

# Train and Test the model 

In [9]:
EPOCH     = 2
L2_factor = 0.0005
L1_factor = 0.001
OPTIMIZER = optim.SGD(model.parameters(), lr=0.025, momentum=0.9,weight_decay=L2_factor)

for epoch in range(EPOCH):
    print("EPOCH:", epoch)
    train_loss.train_loss_calc(model,device,train_loader=trainloader,optimizer=OPTIMIZER,epoch=EPOCH,factor=L1_factor)
    a, b = test_loss.test_loss_calc(model,device,test_loader=testloader)

  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 0


Epoch = 2 Train Loss = 1.1561788320541382 Batch Id = 390 Train Accuracy = 46.87: 100%|██████████| 391/391 [00:23<00:00, 16.39it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 143.8381, Test Accuracy: 74.19

EPOCH: 1


Epoch = 2 Train Loss = 0.9637211561203003 Batch Id = 390 Train Accuracy = 63.03: 100%|██████████| 391/391 [00:24<00:00, 16.26it/s]



Test set: Average loss: 128.1587, Test Accuracy: 80.92

